# Determining Best Combination of Preprocessing Techniques on Test Set

## Loading Libraries

In [1]:
%load_ext autoreload
%autoreload 2

# internal imports
import helpers as HL
import glove_module as GV
import neural_nets as NN
import tokenizing as TO


Using TensorFlow backend.
/Users/HeddaVik/anaconda/lib/python3.6/importlib/_bootstrap.py:205: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


## Create the word embeddings using the created gensim-.txt file.

In [ ]:
#Pick one, the higher dimension, the better result and longer computational time. 
global_vectors=HL.get_global_vectors(200)

## Creating corpus:
In addition to the acutal corpus, some additional information is needed 

In [2]:
full_corpus, nr_pos_tweets, nr_neg_tweets, total_training_tweets=HL.get_corpus(test=True)

## Picking the neural net
At this stage, we want to use the simple neural net

In [ ]:
neural_nets=[NN.basic_model_adam]

## Initializing variables to apply all preprocessing techniques:

In [ ]:
#Initializing vectors:
corpuses=[]
corpuses.append(full_corpus)

In [ ]:
#Defining names of corpuses: 
names=['original_corpus','SH_corpus','SHM_corpus','H_corpus','HK_corpus','PS_corpus','NS__corpus','OS_corpus','N_corpus','NM_corpus','ST_corpus','SP_corpus','E_corpus','SN_corpus','RS_corpus','EX_corpus','N-2_corpus','N-3_corpus','N-4_corpus']

In [ ]:
#Defining inputs to preprocessing function: 
inputs=[{'hashtag': True, 'segmentation_hash': True},
        {'hashtag':True,'segmentation_hash': True,'hashtag_mention':True},
        {'hearts':True},
        {'hugs_and_kisses':True},
        {'pos_smilies':True},
        {'neg_smilies':True},
        {'other_smilies':True},
        {'numbers':True},
        {'numbers':True,'number_mention':True},
        {'stemming':True},
        {'spelling':True},#Warning: When True, it takes app  2.5 h on test set. Recomended to always set to false 
        {'elongation':True},
        {'set_to_not':True},
        {'remove_signs':True},
        {'exclamation':True}]

## Applying all preprocessing techniques to the original corpus: 

In [ ]:

for input_ in inputs: 
        corpus=TO.preprocess_corpus(full_corpus, **input_)
        corpuses.append(corpus)
        

In [ ]:
ns=[2,3,4]
for n in ns: 
    corpus=HL.creating_n_grams_corpus(n,full_corpus)
    corpuses.append(corpus)

## Testing all preprocessing techniques: 

In [ ]:
accuracies=[]
stds=[]

for corpus in corpuses: 
    model_score=GV.classify_with_neural_networks(neural_nets, global_vectors, corpus, total_training_tweets, nr_pos_tweets, epochs=100, n_folds=3)
    accuracies.append(model_score[0][0])
    stds.append(model_score[0][1])

## Want to determine which preprocessing techniques that improved the accuracy, and keep them: 

In [ ]:
corpuses_1=[]
names_1=[]
stds_1=[]
acc_1=[]
print('The original corpus gave accuracy of: ',accuracies[0], 'std:', stds[0],'\n')
for i in range(1,len(accuracies)):
    if accuracies[i]>=accuracies[0]:
        corpuses_1.append(corpuses[i])
        names_1.append(names[i])
        stds_1.append(stds[i])
        acc_1.append(accuracies[i])
        print('IMPROVED:  ',names[i],', score:',accuracies[i],'std:',stds[i])
    else:
        print('Not better:',names[i],', score:',accuracies[i],'std:',stds[i])
        

In [ ]:
# We do not keep SH, as SHM is better. 
# We do not keep N as NM is better 
# We do not keep SP due to time  (158.26 min)
# We do not keep techniques that imporved less than 0.05 percentage points. 

## Apply all techniques that contributed positively

In [5]:
input_={'hashtag': True, 'segmentation_hash': True, 'hashtag_mention':True,
        'hearts':True,'hugs_and_kisses':True,'elongation':True, 'set_to_not':True}


In [6]:
#Applying the techniques that we have decided to keep:
best_prepr_corpus=TO.preprocess_corpus(full_corpus, **input_)
best_corpus=HL.creating_n_grams_corpus(2,best_prepr_corpus)

Reading twitter - 1grams ...
Reading twitter - 2grams ...
Time in min before starting first for loop: 0.16130030155181885
Time in min after 1  tweets: 0.16163189808527628
Time in min after 25001  tweets: 0.5065584659576416
Time in min after 50001  tweets: 0.7630061666170757
Time in min after 75001  tweets: 1.015561584631602
Time in min after 100001  tweets: 1.3070504864056904
Time in min after 125001  tweets: 1.5595041155815124
Time in min after 150001  tweets: 1.7984455982844034
Time in min after 175001  tweets: 2.0378227670987448
Time in min after 200001  tweets: 2.318515717983246
Time in min total: 2.412290334701538


In [ ]:
#Cross validating:
model_score=GV.classify_with_neural_networks(neural_nets, global_vectors, corpus, 
                                             total_training_tweets, nr_pos_tweets, epochs=100, n_folds=3)

In [ ]:
print('Accuracy:',  model_score[0][0], 'std:',model_score[0][1],'\n' )

## Dynamic stopword list

In [7]:
mindfs= [0,2,3,5,10,20, 40, 60,100,140,200]
maxdfs=[0.8] 
accuracies_stop=[]
stds_stop=[]
stop_lens=[]
vocabs=[]


for max_ in maxdfs:
    for min_ in mindfs: 
        stopwords, vocab= TO.get_dynamic_stopwords(best_corpus, MinDf=min_, MaxDf=max_)
        stopword_corpus=TO.remove_stopwords(best_corpus, stopwords)
        model_score=GV.classify_with_neural_networks(neural_nets, global_vectors, stopword_corpus, total_training_tweets, nr_pos_tweets, epochs=100, n_folds=3)
        accuracies_stop.append(model_score[0][0])
        stds_stop.append(model_score[0][1])
        stop_lens.append(len(stopwords))
        vocabs.append(vocab)


In [ ]:
for i in range(len(stop_lens)):
    print('min', mindfs[i], 'accuracy', accuracies_stop[i],'std',stds_stop[i],  '\n')
for i in range(len(stop_lens)):
    print('number of stop words', stop_lens[i], 'lenght of vocabolary', len(vocabs[i]),'sum for checking',stop_lens[i]+len(vocabs[i]),  '\n')